# Building Agents with Praval: A Hands-On Tutorial

Welcome to this interactive tutorial on building AI agents with Praval! This notebook will guide you step-by-step through creating intelligent, collaborative agent systems.

## What You'll Learn

1. **Your First Agent** - Understanding agent identity and the `@agent` decorator
2. **Agent Communication** - How agents talk to each other via spores
3. **Using Chat & LLMs** - Integrating AI models into your agents
4. **Tools for Agents** - Extending agent capabilities with custom tools
5. **Memory-Enabled Agents** - Building agents that remember and learn
6. **Specialist Collaboration** - Creating teams of specialized agents
7. **Knowledge Base Integration** - Building expert agents with domain knowledge

## Praval Philosophy

> *Like coral reefs in nature, complex intelligence emerges from the collaboration of simple, specialized agents.*

Instead of building monolithic AI systems, Praval lets you create ecosystems of specialized agents that collaborate naturally. Each agent has a clear **identity** and **purpose**, making your code easier to understand, test, and extend.

Let's get started!

## Setup

First, let's set up our environment and import the Praval framework.

In [1]:
# Setup: Add src to path if needed
import sys
import os
from pathlib import Path

# Add parent directory to path to import praval
parent_dir = Path(os.getcwd()).parent / 'src'
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# Try to load .env file from project root
project_root = Path(os.getcwd()).parent
env_file = project_root / '.env'

if env_file.exists():
    print(f"📄 Found .env file at: {env_file}")
    try:
        from dotenv import load_dotenv
        load_dotenv(env_file)
        print("✅ Loaded environment variables from .env file")
    except ImportError:
        print("⚠️ python-dotenv not installed. Install with: pip install python-dotenv")
        print("   For now, you can set API keys manually in the notebook.")
else:
    print(f"💡 No .env file found at: {env_file}")
    print("   You can create one by copying .env.example:")
    print(f"   cp {project_root}/.env.example {project_root}/.env")
    print("   Then add your API keys to the .env file")

# Import Praval components
from praval import agent, chat, broadcast, start_agents, tool
from praval import get_agent_info, MEMORY_AVAILABLE, TOOLS_AVAILABLE

print("\n✅ Praval imported successfully!")
print(f"📦 Memory available: {MEMORY_AVAILABLE}")
print(f"🔧 Tools available: {TOOLS_AVAILABLE}")

print("\n💡 Praval now automatically handles agent re-registration in notebooks!")
print("   You can re-run cells without worrying about duplicate outputs.")

📄 Found .env file at: /Users/rajesh/Github/praval/.env
✅ Loaded environment variables from .env file

✅ Praval imported successfully!
📦 Memory available: True
🔧 Tools available: True

💡 Praval now automatically handles agent re-registration in notebooks!
   You can re-run cells without worrying about duplicate outputs.


In [2]:
# Check for API keys (optional - examples will work with fallbacks)
import os

# First, let's see what environment variables are visible
print("🔍 Checking environment for API keys...\n")

# Check for API keys
has_openai = bool(os.getenv('OPENAI_API_KEY'))
has_anthropic = bool(os.getenv('ANTHROPIC_API_KEY'))
has_cohere = bool(os.getenv('COHERE_API_KEY'))

print("API Key Status:")
print(f"  OpenAI: {'✅ Found' if has_openai else '❌ Not found'}")
print(f"  Anthropic: {'✅ Found' if has_anthropic else '❌ Not found'}")
print(f"  Cohere: {'✅ Found' if has_cohere else '❌ Not found'}")

if not any([has_openai, has_anthropic, has_cohere]):
    print("\n⚠️ No API keys detected in Jupyter environment")
    print("\n💡 To set API keys in Jupyter:")
    print("   1. Option A - Set in notebook:")
    print("      import os")
    print("      os.environ['OPENAI_API_KEY'] = 'your-key-here'")
    print("\n   2. Option B - Set in shell before launching Jupyter:")
    print("      export OPENAI_API_KEY='your-key-here'")
    print("      jupyter lab")
    print("\n   3. Option C - Use a .env file with python-dotenv:")
    print("      pip install python-dotenv")
    print("      # Create .env file with your keys")
    print("      from dotenv import load_dotenv")
    print("      load_dotenv()")
    print("\n✨ Don't worry - all examples have fallback responses!")
else:
    print("\n✅ At least one API provider is configured!")
    print("💬 You'll be able to use the chat() function with LLMs")

print("\n💡 Note: This tutorial works with or without API keys!")

🔍 Checking environment for API keys...

API Key Status:
  OpenAI: ✅ Found
  Anthropic: ❌ Not found
  Cohere: ❌ Not found

✅ At least one API provider is configured!
💬 You'll be able to use the chat() function with LLMs

💡 Note: This tutorial works with or without API keys!


In [3]:
# Note: Praval v0.7.10+ automatically prevents duplicate agent subscriptions!
#
# When you re-run a cell that defines an agent, Praval now automatically
# replaces the old handler instead of adding a duplicate. This means:
#
# ✅ No duplicate outputs when re-running cells
# ✅ Interactive development works smoothly in Jupyter
# ✅ No need for manual cleanup or reset
#
# This is thanks to the fix in reef.py where Channel.subscribe() now
# uses replace=True by default, perfect for notebook workflows!

print("💡 Tip: You can re-run any agent cell as many times as you want")
print("   Praval automatically prevents duplicate subscriptions!")

💡 Tip: You can re-run any agent cell as many times as you want
   Praval automatically prevents duplicate subscriptions!


---

# Chapter 1: Your First Agent

## What is an Agent in Praval?

In Praval, an agent is not just a function that processes data. An agent has an **identity** - it knows who it is and what its purpose is. This identity-driven approach makes agents more consistent, predictable, and easier to work with.

### Key Concepts:
- **Identity over Behavior**: Define what the agent IS, not just what it DOES
- **Decorator Pattern**: Transform functions into agents with `@agent`
- **System Message**: The agent's personality comes from its docstring
- **Autonomy**: Agents act independently based on their identity

Let's create your first agent!

In [4]:
# OPTIONAL: Set your API key here for this session
# Uncomment ONE of the lines below and add your API key

# For OpenAI:
# os.environ['OPENAI_API_KEY'] = 'sk-...'

# For Anthropic:
# os.environ['ANTHROPIC_API_KEY'] = 'sk-ant-...'

# For Cohere:
# os.environ['COHERE_API_KEY'] = '...'

# After uncommenting and setting your key, run this cell
# Then re-run the API key check cell above to verify

print("💡 Tip: Set your API key above, then re-run the previous cell to verify")

💡 Tip: Set your API key above, then re-run the previous cell to verify


### Setting API Keys (Optional)

If you want to use real LLM responses instead of fallbacks, you can set your API key directly here:

**Note**: Uncomment and run the cell below to set your API key for this session.

In [5]:
# Your First Agent: A Simple Greeter

@agent("greeter")
def simple_greeter(spore):
    """
    I am a friendly greeter who welcomes people warmly.
    I always make people feel welcomed and valued.
    """
    name = spore.knowledge.get("name", "friend")
    
    greeting = f"Hello, {name}! Welcome to the world of Praval agents!"
    print(f"👋 Greeter: {greeting}")
    
    return {"greeting": greeting, "agent": "greeter"}

# Let's see what we created
info = get_agent_info(simple_greeter)
print(f"\n📋 Agent Info:")
print(f"  Name: {info['name']}")
print(f"  Channel: {info['channel']}")
print(f"  Auto-broadcast: {info['auto_broadcast']}")


📋 Agent Info:
  Name: greeter
  Channel: greeter_channel
  Auto-broadcast: True


### Understanding the Code

Let's break down what just happened:

1. **`@agent("greeter")`**: This decorator transforms the function into an agent named "greeter"
2. **Docstring**: The agent's identity and personality - "I am a friendly greeter..."
3. **`spore` parameter**: Agents receive messages (called "spores") containing knowledge
4. **`spore.knowledge`**: A dictionary with data sent to the agent
5. **Return value**: Agents can return data that gets auto-broadcast to other agents

Now let's run our agent!

In [8]:
# Running your agent with start_agents()

print("🚀 Starting greeter agent...\n")

result = start_agents(
    simple_greeter,
    initial_data={"name": "Alice"}
)

print(f"\n✅ Result: {result}")

🚀 Starting greeter agent...


✅ Result: e000eb35-eb1a-41ad-9463-82e6bd60b358
👋 Greeter: Hello, Alice! Welcome to the world of Praval agents!


### Agent Identity and System Messages

The docstring you write becomes the agent's **system message** - its core identity. This is crucial when agents use LLMs, as it guides their behavior.

Let's see the difference between a formal agent and a casual agent:

In [10]:
# Two agents with different identities

@agent("formal_assistant")
def formal_agent(spore):
    """
    I am a formal, professional assistant who communicates with 
    precision and maintains proper business etiquette at all times.
    """
    task = spore.knowledge.get("task", "assist you")
    response = f"Good day. I shall proceed to {task} with utmost professionalism."
    print(f"🎩 Formal Assistant: {response}")
    return {"response": response}

@agent("casual_buddy")
def casual_agent(spore):
    """
    I'm your laid-back buddy who keeps things fun and friendly.
    I love using casual language and making people smile!
    """
    task = spore.knowledge.get("task", "help out")
    response = f"Hey there! Let's totally {task} together - this'll be awesome!"
    print(f"😎 Casual Buddy: {response}")
    return {"response": response}

# Run both with the same task
print("Same task, different identities:\n")
start_agents(formal_agent, initial_data={"task": "analyze the quarterly report"})
start_agents(casual_agent, initial_data={"task": "analyze the quarterly report"})

Same task, different identities:

👋 Greeter: Hello, friend! Welcome to the world of Praval agents!
😎 Casual Buddy: Hey there! Let's totally analyze the quarterly report together - this'll be awesome!
👋 Greeter: Hello, friend! Welcome to the world of Praval agents!
🎩 Formal Assistant: Good day. I shall proceed to analyze the quarterly report with utmost professionalism.


'9d076a5a-be73-43eb-8345-298d35e3ed32'

😎 Casual Buddy: Hey there! Let's totally analyze the quarterly report together - this'll be awesome!🎩 Formal Assistant: Good day. I shall proceed to analyze the quarterly report with utmost professionalism.



### Try It Yourself!

Create your own agent with a unique identity. Some ideas:
- A poet who speaks in verse
- A scientist who explains everything technically
- A motivational coach who encourages people
- A detective who analyzes clues

In [ ]:
# Exercise: Create Your Own Agent!

@agent("your_agent_name")
def your_custom_agent(spore):
    """
    Write your agent's identity here - who are they?
    What's their personality? How do they communicate?
    """
    # Your code here
    message = spore.knowledge.get("message", "Hello!")
    
    # Process the message according to your agent's identity
    response = f"Your agent's response to: {message}"
    print(response)
    
    return {"response": response}

# Test your agent
# start_agents(your_custom_agent, initial_data={"message": "Tell me about yourself!"})

---

# Chapter 2: Agent Communication

## How Agents Talk to Each Other

In Praval, agents communicate through structured messages called **spores**. Think of spores like messages in a group chat - agents can broadcast information and respond to messages that interest them.

### Key Concepts:
- **Spores**: Structured messages containing knowledge
- **Channels**: Named communication streams (like chat rooms)
- **`broadcast()`**: Send messages to other agents
- **`responds_to`**: Filter which messages an agent handles
- **Message Types**: Organize communication by purpose

Let's see agents in conversation!

In [ ]:
# Two agents having a conversation
# IMPORTANT: Both agents need to subscribe to a SHARED channel for communication

@agent("questioner", channel="conversation", responds_to=["start_conversation"])
def question_agent(spore):
    """
    I am naturally curious and love asking thoughtful questions
    to spark interesting discussions.
    """
    topic = spore.knowledge.get("topic", "life")
    
    question = f"What do you find most interesting about {topic}?"
    print(f"🤔 Questioner: {question}")
    
    # Broadcast to the shared "conversation" channel
    broadcast({
        "type": "question_asked",
        "topic": topic,
        "question": question
    }, channel="conversation")
    
    return {"question": question}

@agent("responder", channel="conversation", responds_to=["question_asked"])
def answer_agent(spore):
    """
    I am thoughtful and enjoy providing insightful answers
    that encourage deeper thinking.
    """
    question = spore.knowledge.get("question")
    topic = spore.knowledge.get("topic")
    
    answer = f"About {topic}, I find the interconnections fascinating - how simple elements combine to create complexity."
    print(f"💡 Responder: {answer}")
    
    return {"answer": answer}

# Start the conversation
print("🎭 Starting agent conversation...\n")
start_agents(
    question_agent,
    answer_agent,
    initial_data={"type": "start_conversation", "topic": "artificial intelligence"}
)

### Understanding Agent Communication

Here's what happened:

1. **`responds_to=["start_conversation"]`**: Questioner only acts when it sees this message type
2. **`broadcast({...})`**: Questioner sends a message to all listening agents
3. **`responds_to=["question_asked"]`**: Responder waits for questions
4. **Self-organization**: No central coordinator - agents coordinate naturally!

Let's create a more complex conversation chain:

In [ ]:
# A conversation chain with multiple agents
# All agents use the shared "conversation" channel

@agent("observer", channel="conversation", responds_to=["question_asked"])
def observing_agent(spore):
    """
    I observe conversations and add interesting observations
    and alternative perspectives.
    """
    question = spore.knowledge.get("question")
    topic = spore.knowledge.get("topic")
    
    observation = f"I notice this question about {topic} touches on both practical and philosophical dimensions."
    print(f"👁️ Observer: {observation}")
    
    # Add another layer to the conversation
    broadcast({
        "type": "observation_shared",
        "topic": topic,
        "observation": observation
    }, channel="conversation")
    
    return {"observation": observation}

@agent("synthesizer", channel="conversation", responds_to=["observation_shared"])
def synthesis_agent(spore):
    """
    I synthesize different perspectives into coherent insights
    that capture the essence of the discussion.
    """
    observation = spore.knowledge.get("observation")
    topic = spore.knowledge.get("topic")
    
    synthesis = f"Synthesizing the discussion on {topic}: Multiple perspectives reveal deeper understanding than any single viewpoint."
    print(f"🎯 Synthesizer: {synthesis}")
    
    return {"synthesis": synthesis}

# Run all four agents in a conversation
print("🎭 Multi-agent conversation chain...\n")
start_agents(
    question_agent,
    answer_agent,
    observing_agent,
    synthesis_agent,
    initial_data={"type": "start_conversation", "topic": "creativity"}
)

### Message Flow Visualization

```
Initial Data → Questioner
                    ↓
              [question_asked]
                    ↓
         ┌──────────┴──────────┐
         ↓                     ↓
    Responder              Observer
                               ↓
                      [observation_shared]
                               ↓
                          Synthesizer
```

This is the power of Praval - agents coordinate naturally through message types!

### Try It Yourself!

Create a conversation between 3 agents:
1. A storyteller who starts a story
2. A plot-twister who adds an unexpected twist
3. A resolver who provides a satisfying ending

In [ ]:
# Exercise: Build a 3-Agent Story Chain

@agent("storyteller", responds_to=["begin_story"])
def story_starter(spore):
    """
    I create engaging story openings that set the scene.
    """
    # Your code here
    pass

@agent("plot_twister", responds_to=["story_started"])
def add_twist(spore):
    """
    I add unexpected plot twists that surprise readers.
    """
    # Your code here
    pass

@agent("resolver", responds_to=["twist_added"])
def resolve_story(spore):
    """
    I create satisfying story conclusions.
    """
    # Your code here
    pass

# Test your story chain
# start_agents(story_starter, add_twist, resolve_story, 
#              initial_data={"type": "begin_story", "genre": "mystery"})

---

# Chapter 3: Using Chat & LLMs

## Integrating AI Models into Your Agents

So far, our agents have used pre-defined responses. Now let's make them truly intelligent by connecting them to Large Language Models (LLMs)!

### Key Concepts:
- **`chat()` function**: Quick access to LLMs from within agents
- **Context awareness**: The agent's identity guides the LLM
- **Error handling**: Graceful fallbacks when LLMs aren't available
- **Prompt engineering**: How to phrase requests to LLMs

Let's create an intelligent agent:

In [ ]:
# An intelligent agent using LLMs

@agent("poet")
def poetic_agent(spore):
    """
    I am a poet who transforms ordinary thoughts into beautiful verse.
    I see the world through metaphor and rhythm.
    """
    topic = spore.knowledge.get("topic", "nature")
    
    print(f"📜 Poet: Crafting verse about {topic}...")
    
    try:
        # Use chat() to interact with LLM
        # The agent's identity (from docstring) guides the response
        poem = chat(f"""
        Write a short, beautiful poem (4-6 lines) about {topic}.
        Use vivid imagery and metaphor.
        """)
        
        print(f"\n{poem}\n")
        return {"poem": poem, "topic": topic}
        
    except Exception as e:
        # Fallback when LLM is not available
        fallback = f"In {topic}'s embrace, we find\nBeauty woven, undefined\nNature speaks in whispered rhyme\nEchoing through space and time"
        print(f"\n{fallback}\n")
        print(f"⚠️ Using fallback (LLM not available: {e})")
        return {"poem": fallback, "topic": topic}

# Test the poet
start_agents(poetic_agent, initial_data={"topic": "starlight"})

### Understanding `chat()`

The `chat()` function:
1. Can only be called from within an `@agent` decorated function
2. Automatically uses the agent's system message (identity) as context
3. Connects to the best available LLM provider (OpenAI, Anthropic, Cohere)
4. Returns a string response

Let's see how agent identity influences responses:

In [ ]:
# Same task, different identities, different responses

@agent("scientist")
def scientific_explainer(spore):
    """
    I am a scientist who explains phenomena using rigorous scientific principles,
    technical terminology, and evidence-based reasoning.
    """
    topic = spore.knowledge.get("topic")
    
    try:
        explanation = chat(f"Explain {topic} in 2-3 sentences.")
        print(f"🔬 Scientist: {explanation}\n")
    except Exception as e:
        print(f"🔬 Scientist: {topic} can be understood through systematic observation and empirical analysis.\n")
    
    return {"explanation": "scientific"}

@agent("storyteller")
def story_explainer(spore):
    """
    I am a storyteller who explains concepts through engaging narratives,
    metaphors, and relatable human experiences.
    """
    topic = spore.knowledge.get("topic")
    
    try:
        explanation = chat(f"Explain {topic} in 2-3 sentences.")
        print(f"📚 Storyteller: {explanation}\n")
    except Exception as e:
        print(f"📚 Storyteller: Imagine {topic} as a journey that we all take together...\n")
    
    return {"explanation": "narrative"}

# Same prompt, different results due to agent identity
print("🎭 Two explanations of the same concept:\n")
start_agents(scientific_explainer, initial_data={"topic": "gravity"})
start_agents(story_explainer, initial_data={"topic": "gravity"})

### Error Handling Best Practices

Always provide fallbacks for when LLMs are unavailable:

In [ ]:
# Robust agent with good error handling

@agent("advisor")
def helpful_advisor(spore):
    """
    I am a helpful advisor who provides practical guidance
    and actionable suggestions.
    """
    situation = spore.knowledge.get("situation", "facing a challenge")
    
    print(f"💼 Advisor: Considering your situation...\n")
    
    try:
        advice = chat(f"""
        Someone is {situation}.
        Provide 3 practical, actionable pieces of advice.
        Be encouraging and specific.
        """)
        print(advice)
        return {"advice": advice, "source": "llm"}
        
    except Exception as e:
        # Structured fallback advice
        fallback_advice = f"""
        For {situation}, I recommend:
        
        1. Break it down: Divide the challenge into smaller, manageable steps
        2. Seek input: Get perspectives from others who've faced similar situations
        3. Take action: Start with the smallest step you can take today
        
        Remember: Progress comes from consistent, small actions!
        """
        print(fallback_advice)
        print(f"\n⚠️ Using fallback advice (LLM unavailable)")
        return {"advice": fallback_advice, "source": "fallback"}

# Test the advisor
start_agents(helpful_advisor, initial_data={"situation": "learning a new skill"})

### Try It Yourself!

Create an agent that uses `chat()` to generate creative content based on its unique identity.

In [ ]:
# Exercise: Create an Intelligent Agent

@agent("your_intelligent_agent")
def your_llm_agent(spore):
    """
    Define your agent's identity here.
    This identity will guide how the LLM responds!
    """
    # Get input from the spore
    user_input = spore.knowledge.get("input", "Hello!")
    
    try:
        # Use chat() to generate intelligent responses
        response = chat(f"""
        Your prompt to the LLM here.
        User said: {user_input}
        """)
        
        print(f"Your agent: {response}")
        return {"response": response}
        
    except Exception as e:
        # Always provide a fallback!
        fallback = "Your fallback response here"
        print(f"Your agent: {fallback}")
        return {"response": fallback}

# Test your agent
# start_agents(your_llm_agent, initial_data={"input": "Tell me something interesting!"})

---

# Chapter 4: Tools for Agents

## Extending Agent Capabilities

Tools give agents structured capabilities - specific functions they can use to accomplish tasks. Think of tools as the agent's hands and equipment.

### Key Concepts:
- **`@tool` decorator**: Define reusable functions as tools
- **Tool ownership**: Tools can belong to specific agents or be shared
- **Tool discovery**: Agents automatically discover their tools
- **Structured capabilities**: Tools provide predictable, testable functionality

Let's create an agent with tools:

In [ ]:
# Define tools first

@tool("calculator", owned_by="mathematician", category="math")
def calculate(expression: str) -> str:
    """
    Safely evaluate mathematical expressions.
    
    Args:
        expression: A mathematical expression like "2 + 2" or "10 * 5"
    
    Returns:
        The result as a string
    """
    try:
        # Safe evaluation (only math operations)
        result = eval(expression, {"__builtins__": {}}, {})
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error calculating {expression}: {e}"

@tool("factorize", owned_by="mathematician", category="math")
def find_factors(number: int) -> str:
    """
    Find all factors of a number.
    
    Args:
        number: The number to factorize
    
    Returns:
        A formatted string with all factors
    """
    factors = [i for i in range(1, abs(number) + 1) if number % i == 0]
    return f"Factors of {number}: {factors}"

print("🔧 Tools defined successfully!")

In [ ]:
# Create an agent that uses these tools

@agent("mathematician")
def math_expert(spore):
    """
    I am a mathematician who solves problems using logical reasoning
    and mathematical tools. I explain my process clearly.
    """
    problem = spore.knowledge.get("problem", "2 + 2")
    
    print(f"🔢 Mathematician: Working on '{problem}'...\n")
    
    # List available tools
    tools = math_expert.list_tools()
    print(f"📦 Available tools: {tools}")
    
    # Use the calculator tool
    if "calculator" in tools:
        calc_tool = math_expert.get_tool("calculator")
        if calc_tool:
            result = calc_tool(problem)
            print(f"\n🔢 Result: {result}")
    
    # If the problem includes factorization
    if "factor" in problem.lower():
        try:
            # Extract number from problem
            number = int(''.join(filter(str.isdigit, problem)))
            if math_expert.has_tool("factorize"):
                factor_tool = math_expert.get_tool("factorize")
                factors = factor_tool(number)
                print(f"\n🔢 Factors: {factors}")
        except:
            pass
    
    return {"problem": problem, "tools_used": tools}

# Test the mathematician with different problems
print("Testing mathematician agent:\n")
start_agents(math_expert, initial_data={"problem": "15 * 7"})
print("\n" + "="*60 + "\n")
start_agents(math_expert, initial_data={"problem": "find factors of 24"})

### Understanding Tools

Key points about tools:

1. **Definition**: Use `@tool(name, owned_by=agent_name)` to create tools
2. **Auto-registration**: Tools owned by an agent are automatically available
3. **Discovery**: Use `agent.list_tools()` to see what's available
4. **Access**: Use `agent.get_tool(name)` to get a tool function
5. **Check**: Use `agent.has_tool(name)` to check availability

Let's create a more sophisticated agent with multiple tools:

In [ ]:
# A research agent with multiple tools

@tool("search_database", owned_by="researcher", category="research")
def search_db(query: str) -> str:
    """
    Simulate searching a research database.
    """
    # In reality, this would query a real database
    mock_results = {
        "ai": "Found 1,247 papers on artificial intelligence",
        "quantum": "Found 832 papers on quantum computing",
        "biology": "Found 2,156 papers on molecular biology"
    }
    
    for key, result in mock_results.items():
        if key in query.lower():
            return result
    
    return f"Found 42 papers related to '{query}'"

@tool("summarize", owned_by="researcher", category="research")
def summarize_text(text: str, length: str = "short") -> str:
    """
    Summarize text to specified length.
    """
    # Simple summarization (in reality, use LLM or extractive summarization)
    words = text.split()
    max_words = 20 if length == "short" else 50
    summary = ' '.join(words[:max_words])
    return f"Summary: {summary}..."

@tool("cite", owned_by="researcher", category="research")
def generate_citation(title: str, author: str = "Unknown") -> str:
    """
    Generate a citation in APA format.
    """
    return f"{author}. (2024). {title}. Journal of Research."

@agent("researcher")
def research_assistant(spore):
    """
    I am a research assistant who helps with literature reviews,
    finding relevant papers, and managing citations.
    """
    topic = spore.knowledge.get("topic", "general research")
    
    print(f"📚 Researcher: Investigating '{topic}'...\n")
    
    # Show all tools
    tools = research_assistant.list_tools()
    print(f"🔧 My research tools: {', '.join(tools)}\n")
    
    # Use search tool
    if research_assistant.has_tool("search_database"):
        search = research_assistant.get_tool("search_database")
        results = search(topic)
        print(f"🔍 {results}")
    
    # Use citation tool
    if research_assistant.has_tool("cite"):
        cite = research_assistant.get_tool("cite")
        citation = cite(f"Advances in {topic}", "Smith et al.")
        print(f"\n📝 Example citation: {citation}")
    
    return {"topic": topic, "tools_available": len(tools)}

# Test the research assistant
start_agents(research_assistant, initial_data={"topic": "quantum computing"})

### Try It Yourself!

Create an agent with custom tools. Some ideas:
- A chef with recipe tools
- A weather agent with forecasting tools
- A translator with language tools
- A data analyst with visualization tools

In [ ]:
# Exercise: Create an Agent with Custom Tools

# Define your tools
@tool("your_tool_1", owned_by="your_agent", category="your_category")
def your_first_tool(param: str) -> str:
    """
    Describe what this tool does.
    """
    # Your tool logic here
    return f"Tool 1 processed: {param}"

@tool("your_tool_2", owned_by="your_agent", category="your_category")
def your_second_tool(param: str) -> str:
    """
    Describe what this tool does.
    """
    # Your tool logic here
    return f"Tool 2 processed: {param}"

# Create your agent
@agent("your_agent")
def your_tool_agent(spore):
    """
    Define your agent's identity and purpose.
    """
    # Use your tools here
    tools = your_tool_agent.list_tools()
    print(f"Available tools: {tools}")
    
    # Your agent logic using tools
    
    return {"tools_used": tools}

# Test your agent
# start_agents(your_tool_agent, initial_data={"task": "do something"})

---

# Chapter 5: Memory-Enabled Agents

## Building Agents That Remember and Learn

Memory transforms agents from stateless functions into entities that learn from experience and build relationships over time.

### Key Concepts:
- **`memory=True`**: Enable memory for an agent
- **`remember()`**: Store information for later
- **`recall()`**: Retrieve relevant memories semantically
- **Persistence**: Memories survive across sessions
- **Semantic search**: Find memories by meaning, not just keywords

Let's create a memory-enabled agent:

In [ ]:
# Check if memory is available
if not MEMORY_AVAILABLE:
    print("⚠️ Memory system not available. Install with:")
    print("   pip install chromadb sentence-transformers")
    print("\nThis chapter will use simplified examples.\n")
else:
    print("✅ Memory system available!\n")

In [ ]:
# A learning agent with memory

@agent("learner", memory=True)
def learning_agent(spore):
    """
    I am a curious learner who remembers everything I learn
    and builds on my past knowledge.
    """
    topic = spore.knowledge.get("topic", "general knowledge")
    fact = spore.knowledge.get("fact")
    
    print(f"🧠 Learner: Learning about '{topic}'...\n")
    
    if MEMORY_AVAILABLE:
        # Check what I already know about this topic
        past_knowledge = learning_agent.recall(topic, limit=3)
        
        if past_knowledge:
            print(f"💭 I recall {len(past_knowledge)} things about {topic}:")
            for i, memory in enumerate(past_knowledge, 1):
                print(f"   {i}. {memory.content[:80]}...")
            print()
        else:
            print(f"🆕 This is my first time learning about {topic}!\n")
        
        # Store new knowledge if provided
        if fact:
            learning_agent.remember(
                f"About {topic}: {fact}",
                importance=0.8,
                memory_type="semantic"
            )
            print(f"📝 Stored new knowledge: {fact}\n")
    else:
        print("(Memory system not available - using demonstration mode)\n")
        if fact:
            print(f"📝 Would store: {fact}\n")
    
    return {
        "topic": topic,
        "memories_found": len(past_knowledge) if MEMORY_AVAILABLE else 0
    }

# Test the learner with multiple facts
facts_to_learn = [
    {"topic": "python", "fact": "Python uses indentation for code blocks"},
    {"topic": "python", "fact": "Python has dynamic typing"},
    {"topic": "python", "fact": "Python was created by Guido van Rossum"},
    {"topic": "praval", "fact": "Praval is inspired by coral reef ecosystems"},
]

print("📚 Teaching the learner agent...\n" + "="*60 + "\n")

for fact_data in facts_to_learn:
    start_agents(learning_agent, initial_data=fact_data)
    print("─" * 60 + "\n")

### Understanding Memory

Memory in Praval works through:

1. **`remember(content, importance=0.5, memory_type="episodic")`**
   - Stores information with semantic meaning
   - `importance`: 0.0 to 1.0, affects retrieval priority
   - `memory_type`: "episodic", "semantic", or "procedural"

2. **`recall(query, limit=5)`**
   - Semantic search for relevant memories
   - Returns most relevant memories first
   - Uses vector similarity, not keyword matching

Let's see memory in a practical application:

In [ ]:
# A teaching assistant that remembers student progress

@agent("tutor", memory=True)
def teaching_tutor(spore):
    """
    I am a patient tutor who remembers each student's progress,
    strengths, and areas for improvement.
    """
    student = spore.knowledge.get("student", "student")
    lesson = spore.knowledge.get("lesson")
    performance = spore.knowledge.get("performance", "good")
    
    print(f"👨‍🏫 Tutor: Teaching {student}...\n")
    
    if MEMORY_AVAILABLE:
        # Recall student's history
        student_history = teaching_tutor.recall(f"student {student}", limit=3)
        
        if student_history:
            print(f"📊 {student}'s learning history ({len(student_history)} sessions):")
            for memory in student_history:
                print(f"   • {memory.content}")
            print()
            teaching_approach = "building on previous knowledge"
        else:
            print(f"🆕 First session with {student}!\n")
            teaching_approach = "starting with fundamentals"
        
        # Record this session
        if lesson:
            session_note = f"Student {student}: {lesson} - Performance: {performance}"
            teaching_tutor.remember(session_note, importance=0.7)
            print(f"📝 Recorded session: {lesson} ({performance})\n")
            print(f"📋 Teaching approach: {teaching_approach}")
    else:
        print("(Memory system not available - using demonstration mode)")
        teaching_approach = "adaptive"
    
    return {
        "student": student,
        "lesson": lesson,
        "approach": teaching_approach
    }

# Simulate teaching sessions
sessions = [
    {"student": "Alice", "lesson": "Variables and data types", "performance": "excellent"},
    {"student": "Alice", "lesson": "Control flow", "performance": "good"},
    {"student": "Bob", "lesson": "Variables and data types", "performance": "needs practice"},
    {"student": "Alice", "lesson": "Functions", "performance": "excellent"},
    {"student": "Bob", "lesson": "Control flow", "performance": "improving"},
]

print("🎓 Tutoring sessions:\n" + "="*60 + "\n")

for session in sessions:
    start_agents(teaching_tutor, initial_data=session)
    print("\n" + "─" * 60 + "\n")

### Conversation Context

Memory also tracks conversation history automatically:

In [ ]:
# Agent that maintains conversation context

@agent("conversationalist", memory=True)
def chat_agent(spore):
    """
    I am a conversationalist who remembers our chat history
    and refers back to previous topics naturally.
    """
    message = spore.knowledge.get("message", "Hello!")
    
    print(f"💬 Conversationalist: Received: '{message}'\n")
    
    if MEMORY_AVAILABLE:
        # Get recent conversation context
        context = chat_agent.get_conversation_context(turns=3)
        
        if context:
            print(f"📜 Recent conversation ({len(context)} turns):")
            for turn in context[-3:]:
                print(f"   User: {turn.get('user', 'N/A')[:50]}...")
                print(f"   Me: {turn.get('agent', 'N/A')[:50]}...")
                print()
    
    response = f"Interesting point about '{message}'! That connects to our earlier discussion."
    print(f"💬 Response: {response}\n")
    
    return {"response": response}

# Have a multi-turn conversation
conversation = [
    "What do you think about AI?",
    "How does it affect creativity?",
    "Can AI and humans collaborate effectively?"
]

print("💬 Multi-turn conversation:\n" + "="*60 + "\n")

for msg in conversation:
    start_agents(chat_agent, initial_data={"message": msg})
    print("─" * 60 + "\n")

### Try It Yourself!

Create a memory-enabled agent that learns from interactions. Some ideas:
- A habit tracker that remembers goals and progress
- A recipe collector that remembers favorite dishes
- A journal agent that tracks thoughts over time
- A recommendation agent that learns preferences

In [ ]:
# Exercise: Create a Memory-Enabled Agent

@agent("your_memory_agent", memory=True)
def your_learning_agent(spore):
    """
    Define an agent that benefits from memory.
    What should it remember? How does it use past experiences?
    """
    # Your agent logic here
    
    if MEMORY_AVAILABLE:
        # Use remember() to store information
        # your_learning_agent.remember("something important", importance=0.8)
        
        # Use recall() to retrieve relevant memories
        # memories = your_learning_agent.recall("query", limit=5)
        
        # Use get_conversation_context() for chat history
        # context = your_learning_agent.get_conversation_context(turns=5)
        pass
    
    return {"status": "learning"}

# Test your agent with multiple interactions
# start_agents(your_learning_agent, initial_data={"interaction": 1})

---

# Chapter 6: Specialist Collaboration

## Creating Teams of Expert Agents

The true power of Praval emerges when specialized agents collaborate. Each agent excels at one thing, and together they solve complex problems.

### Key Concepts:
- **Specialization**: Each agent has one clear purpose
- **Coordination**: Agents work together through message types
- **Emergent intelligence**: Complex behavior from simple interactions
- **No central control**: Agents self-organize naturally

Let's build a team of specialists:

In [ ]:
# A team of specialists: Analysis Pipeline

@agent("collector", responds_to=["start_analysis"])
def data_collector(spore):
    """
    I specialize in gathering and organizing raw information
    for analysis.
    """
    topic = spore.knowledge.get("topic")
    
    print(f"📊 Collector: Gathering data about '{topic}'...\n")
    
    # Simulate data collection
    data = {
        "sources": 15,
        "data_points": 247,
        "categories": ["research", "industry", "trends"],
        "quality_score": 0.87
    }
    
    print(f"✅ Collected {data['data_points']} data points from {data['sources']} sources\n")
    
    broadcast({
        "type": "data_ready",
        "topic": topic,
        "data": data
    })
    
    return data

@agent("analyzer", responds_to=["data_ready"])
def pattern_analyzer(spore):
    """
    I specialize in finding patterns, trends, and insights
    in data.
    """
    topic = spore.knowledge.get("topic")
    data = spore.knowledge.get("data", {})
    
    print(f"🔍 Analyzer: Finding patterns in {topic} data...\n")
    
    # Simulate analysis
    patterns = [
        "Growth trend: 23% increase over period",
        "Seasonal variation detected",
        "Strong correlation with market factors"
    ]
    
    insights = {
        "patterns": patterns,
        "confidence": 0.85,
        "anomalies": 2
    }
    
    print("📈 Key patterns found:")
    for pattern in patterns:
        print(f"   • {pattern}")
    print()
    
    broadcast({
        "type": "analysis_complete",
        "topic": topic,
        "insights": insights
    })
    
    return insights

@agent("evaluator", responds_to=["analysis_complete"])
def insight_evaluator(spore):
    """
    I specialize in evaluating the quality and reliability
    of insights and findings.
    """
    topic = spore.knowledge.get("topic")
    insights = spore.knowledge.get("insights", {})
    
    print(f"⚖️ Evaluator: Assessing reliability of {topic} insights...\n")
    
    # Simulate evaluation
    evaluation = {
        "reliability": "High",
        "confidence_score": insights.get("confidence", 0.8),
        "recommendations": [
            "Insights are well-supported by data",
            "Consider additional validation",
            "Ready for reporting"
        ]
    }
    
    print(f"✅ Reliability: {evaluation['reliability']}")
    print(f"📊 Confidence: {evaluation['confidence_score']:.2%}\n")
    
    broadcast({
        "type": "evaluation_complete",
        "topic": topic,
        "evaluation": evaluation
    })
    
    return evaluation

@agent("reporter", responds_to=["evaluation_complete"])
def final_reporter(spore):
    """
    I specialize in synthesizing findings into clear,
    actionable reports.
    """
    topic = spore.knowledge.get("topic")
    evaluation = spore.knowledge.get("evaluation", {})
    
    print(f"📝 Reporter: Creating final report on {topic}...\n")
    
    report = f"""
    ╔════════════════════════════════════════════════════╗
    ║          ANALYSIS REPORT: {topic.upper()}
    ╠════════════════════════════════════════════════════╣
    ║ Status: ✅ Complete
    ║ Reliability: {evaluation.get('reliability', 'N/A')}
    ║ Confidence: {evaluation.get('confidence_score', 0):.0%}
    ║
    ║ The specialist team has completed comprehensive
    ║ analysis with high-quality results. All findings
    ║ have been validated and are ready for action.
    ╚════════════════════════════════════════════════════╝
    """
    
    print(report)
    
    return {"report": report, "topic": topic}

# Run the specialist team
print("🎯 Specialist Team Analysis Pipeline\n")
print("="*60 + "\n")

start_agents(
    data_collector,
    pattern_analyzer,
    insight_evaluator,
    final_reporter,
    initial_data={"type": "start_analysis", "topic": "market trends"}
)

### Understanding Specialist Collaboration

Notice the workflow:

```
start_analysis → Collector
                     ↓
                 data_ready → Analyzer
                                 ↓
                         analysis_complete → Evaluator
                                                 ↓
                                         evaluation_complete → Reporter
```

Each agent:
1. Has ONE clear responsibility
2. Responds to specific message types
3. Broadcasts its results for others
4. No agent knows about the others - they just do their job!

Let's create a parallel collaboration pattern:

In [ ]:
# Parallel specialists: Creative brainstorming team

@agent("coordinator", responds_to=["start_brainstorm"])
def brainstorm_coordinator(spore):
    """
    I coordinate brainstorming sessions by framing the challenge
    and inviting diverse perspectives.
    """
    challenge = spore.knowledge.get("challenge")
    
    print(f"🎯 Coordinator: Framing challenge: '{challenge}'\n")
    
    # Invite all specialists to contribute in parallel
    broadcast({
        "type": "idea_request",
        "challenge": challenge,
        "perspective": "all"
    })
    
    return {"challenge": challenge}

@agent("creative", responds_to=["idea_request"])
def creative_thinker(spore):
    """
    I specialize in bold, innovative ideas that challenge
    conventional thinking.
    """
    challenge = spore.knowledge.get("challenge")
    
    idea = f"What if we completely reimagined {challenge} using biomimicry principles?"
    print(f"💡 Creative: {idea}\n")
    
    broadcast({
        "type": "idea_shared",
        "specialist": "creative",
        "idea": idea
    })
    
    return {"idea": idea}

@agent("practical", responds_to=["idea_request"])
def practical_thinker(spore):
    """
    I specialize in feasible, implementable solutions
    with clear steps.
    """
    challenge = spore.knowledge.get("challenge")
    
    idea = f"Let's address {challenge} by starting with existing resources and scaling gradually."
    print(f"🔧 Practical: {idea}\n")
    
    broadcast({
        "type": "idea_shared",
        "specialist": "practical",
        "idea": idea
    })
    
    return {"idea": idea}

@agent("critical", responds_to=["idea_request"])
def critical_thinker(spore):
    """
    I specialize in identifying potential issues,
    risks, and areas for improvement.
    """
    challenge = spore.knowledge.get("challenge")
    
    idea = f"Before solving {challenge}, we should validate the underlying assumptions and identify constraints."
    print(f"🤔 Critical: {idea}\n")
    
    broadcast({
        "type": "idea_shared",
        "specialist": "critical",
        "idea": idea
    })
    
    return {"idea": idea}

@agent("synthesizer", responds_to=["idea_shared"])
def idea_synthesizer(spore):
    """
    I collect diverse perspectives and synthesize them
    into a balanced approach.
    """
    specialist = spore.knowledge.get("specialist")
    idea = spore.knowledge.get("idea")
    
    # In a real implementation, this would collect all ideas
    # and synthesize once all specialists have contributed
    print(f"🎯 Synthesizer: Noted {specialist}'s perspective\n")
    
    return {"noted": specialist}

# Run parallel brainstorming
print("🧠 Parallel Brainstorming Team\n")
print("="*60 + "\n")

start_agents(
    brainstorm_coordinator,
    creative_thinker,
    practical_thinker,
    critical_thinker,
    idea_synthesizer,
    initial_data={"type": "start_brainstorm", "challenge": "sustainable urban design"}
)

print("\n💡 All specialists contributed in parallel!")

### Try It Yourself!

Design your own specialist team. Some ideas:
- Product development team (designer, engineer, marketer, tester)
- Content creation team (writer, editor, fact-checker, publisher)
- Customer service team (greeter, problem-solver, escalation, follow-up)
- Learning team (instructor, practice-provider, assessor, encourager)

In [ ]:
# Exercise: Design Your Specialist Team

# Define 3-4 specialist agents that work together
# Each should have:
# - A clear, focused responsibility
# - responds_to specific message types
# - broadcasts results for others

@agent("specialist_1", responds_to=["start_workflow"])
def your_specialist_1(spore):
    """
    I specialize in...
    """
    # Your code here
    pass

# Add more specialists...

# Test your team
# start_agents(specialist_1, specialist_2, specialist_3,
#              initial_data={"type": "start_workflow", "task": "something"})

---

# Chapter 7: Knowledge Base Integration

## Building Expert Agents with Domain Knowledge

Knowledge bases allow agents to have instant access to domain-specific information, making them true experts in their field.

### Key Concepts:
- **`knowledge_base` parameter**: Path to documents/files
- **Auto-indexing**: Documents are automatically embedded
- **Semantic search**: Retrieve relevant information by meaning
- **RAG pattern**: Retrieval-Augmented Generation

Let's create an expert agent with a knowledge base:

In [ ]:
# First, create a sample knowledge base
import tempfile
from pathlib import Path

def create_sample_kb():
    """Create a temporary knowledge base for demonstration"""
    kb_dir = Path(tempfile.mkdtemp()) / "agent_knowledge"
    kb_dir.mkdir(parents=True, exist_ok=True)
    
    # Create sample documents
    (kb_dir / "praval_basics.txt").write_text("""
    Praval Agent Framework Basics
    
    Praval is inspired by coral reef ecosystems. Just as coral reefs are built
    by many simple organisms working together, Praval applications are built
    from specialized agents collaborating naturally.
    
    Key principles:
    - Identity over behavior: Define what agents ARE
    - Specialization: Each agent excels at one thing
    - Self-organization: Agents coordinate without central control
    - Emergent intelligence: Complex results from simple interactions
    """)
    
    (kb_dir / "agent_communication.txt").write_text("""
    Agent Communication in Praval
    
    Agents communicate through spores - structured messages containing knowledge.
    Spores are like messages in a group chat. Agents can:
    - Broadcast knowledge to channels
    - Filter messages with responds_to
    - Self-organize through message types
    
    The broadcast() function sends knowledge to all listening agents.
    The responds_to parameter filters which messages an agent handles.
    """)
    
    (kb_dir / "memory_system.txt").write_text("""
    Memory System
    
    Praval agents can have memory enabled with memory=True parameter.
    Memory provides:
    - remember(): Store information for later
    - recall(): Semantic search for relevant memories
    - get_conversation_context(): Access chat history
    
    Memory uses ChromaDB for vector storage and semantic search.
    Memories persist across sessions and scale to millions of entries.
    """)
    
    print(f"📚 Created sample knowledge base at: {kb_dir}")
    return str(kb_dir)

kb_path = create_sample_kb()
print(f"✅ Knowledge base ready!\n")

In [ ]:
# Create an expert agent with knowledge base

@agent("praval_expert", memory=True, knowledge_base=kb_path)
def praval_documentation_expert(spore):
    """
    I am an expert on the Praval framework who can answer questions
    using my comprehensive knowledge base.
    """
    question = spore.knowledge.get("question", "What is Praval?")
    
    print(f"📖 Praval Expert: Answering: '{question}'\n")
    
    if MEMORY_AVAILABLE:
        # Search knowledge base for relevant information
        relevant_docs = praval_documentation_expert.recall(question, limit=2)
        
        if relevant_docs:
            print(f"📚 Found {len(relevant_docs)} relevant documents:\n")
            
            context = ""
            for i, doc in enumerate(relevant_docs, 1):
                print(f"   Document {i}:")
                snippet = doc.content[:150].replace('\n', ' ')
                print(f"   {snippet}...\n")
                context += doc.content + "\n\n"
            
            # Now use this context to answer (would use chat() with real LLM)
            answer = f"""Based on the documentation: {question}
            
The relevant information shows that Praval focuses on agent specialization
and natural collaboration. The key concepts from the knowledge base indicate
that agents work together through structured communication.
"""
        else:
            answer = "I don't have specific information about that in my knowledge base."
    else:
        answer = "(Memory/KB system not available - would search knowledge base here)"
    
    print(f"💡 Answer: {answer}\n")
    
    return {"question": question, "answer": answer}

# Ask the expert questions
questions = [
    "What is Praval?",
    "How do agents communicate?",
    "What is the memory system?"
]

print("🎓 Asking the Praval expert...\n" + "="*60 + "\n")

for q in questions:
    start_agents(praval_documentation_expert, initial_data={"question": q})
    print("─" * 60 + "\n")

### Understanding Knowledge Base Integration

When you specify `knowledge_base="/path/to/docs"`:

1. **Auto-indexing**: All documents are automatically embedded
2. **Semantic search**: `recall()` searches by meaning, not keywords
3. **RAG pattern**: Retrieve relevant docs, then generate responses
4. **Scalable**: Works with thousands of documents

Let's create a practical knowledge base application:

In [ ]:
# A support agent with product documentation

def create_product_kb():
    """Create a sample product knowledge base"""
    kb_dir = Path(tempfile.mkdtemp()) / "product_docs"
    kb_dir.mkdir(parents=True, exist_ok=True)
    
    (kb_dir / "troubleshooting.txt").write_text("""
    Common Issues and Solutions
    
    Login Problems:
    - Clear browser cache and cookies
    - Check that Caps Lock is off
    - Use password reset if needed
    - Verify account is active
    
    Performance Issues:
    - Check internet connection speed
    - Close unnecessary browser tabs
    - Update to latest browser version
    - Clear application cache
    """)
    
    (kb_dir / "features.txt").write_text("""
    Product Features
    
    Dashboard:
    - Real-time analytics and metrics
    - Customizable widgets and layouts
    - Export data to CSV or PDF
    
    Collaboration:
    - Team workspaces and channels
    - @mentions and notifications
    - File sharing and comments
    - Video conferencing integration
    """)
    
    return str(kb_dir)

product_kb = create_product_kb()
print(f"📦 Product knowledge base created\n")

@agent("support_agent", memory=True, knowledge_base=product_kb)
def customer_support(spore):
    """
    I am a helpful customer support agent with access to
    product documentation and troubleshooting guides.
    """
    issue = spore.knowledge.get("issue")
    customer = spore.knowledge.get("customer", "customer")
    
    print(f"🎧 Support Agent: Helping {customer} with: '{issue}'\n")
    
    if MEMORY_AVAILABLE:
        # Search KB for relevant solutions
        solutions = customer_support.recall(issue, limit=2)
        
        if solutions:
            print(f"📚 Found relevant help articles:\n")
            for sol in solutions:
                print(f"   {sol.content[:100]}...\n")
            
            response = f"I found some solutions in our documentation that should help with '{issue}'. Let me guide you through the steps."
        else:
            response = "Let me look into that issue for you. I'll escalate to our technical team."
        
        # Remember this support interaction
        customer_support.remember(
            f"Helped {customer} with {issue}",
            importance=0.7
        )
    else:
        response = "(Would search product documentation here)"
    
    print(f"💬 Response: {response}\n")
    
    return {"customer": customer, "issue": issue, "helped": True}

# Simulate customer support requests
support_requests = [
    {"customer": "Alice", "issue": "can't log in to my account"},
    {"customer": "Bob", "issue": "how do I use the dashboard features?"},
    {"customer": "Alice", "issue": "app is running slow"},
]

print("🎧 Customer Support Sessions\n" + "="*60 + "\n")

for request in support_requests:
    start_agents(customer_support, initial_data=request)
    print("─" * 60 + "\n")

### Try It Yourself!

Create an expert agent with a knowledge base. Some ideas:
- Recipe expert with cooking knowledge
- Fitness coach with workout knowledge
- Legal advisor with law documentation
- Teacher with educational resources

In [ ]:
# Exercise: Create an Expert Agent

# 1. Create your knowledge base
def create_your_kb():
    kb_dir = Path(tempfile.mkdtemp()) / "your_kb"
    kb_dir.mkdir(parents=True, exist_ok=True)
    
    # Create documents with domain knowledge
    (kb_dir / "doc1.txt").write_text("""
    Your domain knowledge here...
    """)
    
    return str(kb_dir)

# your_kb_path = create_your_kb()

# 2. Create your expert agent
# @agent("your_expert", memory=True, knowledge_base=your_kb_path)
# def your_expert_agent(spore):
#     """
#     I am an expert in...
#     """
#     # Use recall() to search your knowledge base
#     # Provide expert answers using the retrieved information
#     pass

# Test your expert
# start_agents(your_expert_agent, initial_data={"question": "..."})

---

# Conclusion: What You've Learned

## 🎉 Congratulations!

You've completed the comprehensive Praval agent-building tutorial. Here's what you can now do:

### Core Skills
✅ **Create agents** with identity-driven design using `@agent`  
✅ **Agent communication** through spores and message types  
✅ **Use LLMs** intelligently with `chat()` and proper fallbacks  
✅ **Extend capabilities** with custom tools using `@tool`  
✅ **Build memory** into agents for learning and continuity  
✅ **Design specialist teams** that collaborate naturally  
✅ **Create expert agents** with domain knowledge bases  

### Key Principles

1. **Identity over Behavior**: Define what agents ARE, not just what they DO
2. **Specialization**: Each agent excels at one thing
3. **Self-Organization**: Agents coordinate without central control
4. **Emergent Intelligence**: Complex behaviors from simple interactions
5. **Composability**: Agents combine naturally through standard interfaces

### The Praval Philosophy

> *Like coral reefs in nature, complex intelligence emerges from the collaboration of simple, specialized agents.*

Instead of building monolithic AI systems, you now build ecosystems of agents that work together naturally.

## Next Steps

### Explore Advanced Examples

Check out these production examples in the `examples/` directory:

1. **VentureLens** (`venturelens.py`)
   - Flagship example: AI business analyzer
   - Multi-agent interview and report generation
   - Professional PDF output

2. **Knowledge Graph Miner** (`knowledge_graph_miner.py`)
   - Advanced multi-threading
   - Concurrent agent processing
   - Complex knowledge structures

3. **RAG Chatbot** (`rag_chatbot.py`)
   - Document processing and embedding
   - Retrieval-augmented generation
   - Conversational AI

4. **Numbered Examples** (`001_*.py` through `011_*.py`)
   - Progressive complexity
   - Specific patterns and techniques
   - Best practices in action

### Design Patterns to Master

1. **Pipeline Pattern**: Sequential specialist processing
2. **Parallel Pattern**: Multiple agents working simultaneously
3. **Hub Pattern**: Coordinator distributing work
4. **Recursive Pattern**: Agents creating sub-agents
5. **Observer Pattern**: Monitoring and logging agents

### Build Your Own

Try building these real-world applications:
- Content creation pipeline (research → write → edit → publish)
- Data analysis team (collect → analyze → visualize → report)
- Customer service system (triage → solve → escalate → follow-up)
- Learning platform (assess → teach → practice → evaluate)

### Resources

- **Documentation**: Check `docs/` directory for detailed specs
- **Philosophy**: Read `praval.md` for design principles
- **Community**: Join discussions and share your agents!

### Remember

The best way to master Praval is to build. Start small, experiment freely, and let emergent intelligence surprise you!

Happy agent building! 🚀

In [ ]:
# Your journey begins here!
# Use this cell to start building your own agent system

print("""
╔════════════════════════════════════════════════════════╗
║                                                        ║
║   🎯 You're ready to build with Praval!               ║
║                                                        ║
║   Start with a simple agent and grow from there.      ║
║   Remember: specialization, identity, collaboration   ║
║                                                        ║
║   The reef awaits your agents! 🪸                      ║
║                                                        ║
╚════════════════════════════════════════════════════════╝
""")